# Io python

In this lesson you will learn about the IO functions of Python.
This will enable you to read and write files as well as 
read and write to serial devices such as the secure bootloader 
you are designing during this course. A series of challenges
follow which will require you to read Python documentation
as well as other online resources. Good luck!

### Challenge Name: echo_int (/embsec/io_python/echo_int)


    1. Read a big-endian short from the serial device
    2. Increment the integer by 1
    3. Send the integer as a big-endian short back to the serial device

Resources:

<https://en.wikipedia.org/wiki/Endianness>

<https://en.wikipedia.org/wiki/Integer_%28computer_science%29>

<https://docs.python.org/3/library/struct.html>




In [15]:
from embsec import Serial
import struct
def echo_int():
    ser = Serial("/embsec/io_python/echo_int")
    # Your code goes here!
    num=ser.read(2)
    numun=struct.unpack(">h",num)[0]
    
    ser.write(struct.pack(">h",numun+1))
#     barray=ser.read()
#     listb=[barray[0]+1]
#     arr=bytearray(listb)
#     print('added one back in byte array form?',arr)
#     #arrstruct=struct.pack('>h',arr)
#     ser.write(arr)
    return(ser.read_until())
echo_int()


b'embsec{echo_int_2f3194887223a2e8}\n'

### Challenge Name: send_file (/embsec/io_python/send_file)



    1. Read 'file.bin' from your local system
    2. Calculate the size
    3. Send the size as a little-endian short
    4. Send the file to serial device

    The serial device expects a little-endian short indicating the size of the
    incoming data and then size bytes of data. The format is represented below:
    
    [ 0x02 ]  [ variable ]
    ---------------------
    | Length |  Data... |
    ---------------------

Resources:

<https://en.wikipedia.org/wiki/Endianness>

<https://en.wikipedia.org/wiki/Integer_%28computer_science%29>

<https://docs.python.org/3/library/struct.html>

<https://docs.python.org/3/tutorial/inputoutput.html>



In [8]:
from embsec import Serial
import struct

def send_file():
    ser = Serial("/embsec/io_python/send_file")
    # Your code goes here!
    with open ('file.bin','r') as file:
        filecon = file.read()
    len_file = len(filecon)
    print('hi')
    len_b = struct.pack("<h",len_file)
    mess = len_b + filecon.encode() + b'\0'
    print(mess)
    ser.write(mess)
    return(ser.read_until())
send_file()


hi
b'\x0f\x00thisisatestfile\x00'


b'embsec{send_file_0729ce0d695e78d7}\n'

### Challenge Name: send_large_file (/embsec/io_python/send_large_file)


    1. Read 'large_file.bin' from your local system
    3. Send the data in frames to the serial device (frame format below)
    4. Send zero-length frame to indicate end of transmission

    The serial device expects that a frame begins with a little-endian short 
    indicating the size of the frame and then frame data. The maximum frame 
    size is 16 bytes. The frame format is represented below:
    
    [ 0x02 ]  [ up to 0xE bytes ]
    ----------------------------
    | Length |      Data...    |
    ----------------------------
    
Resources:
<https://en.wikipedia.org/wiki/Endianness>
<https://en.wikipedia.org/wiki/Integer_%28computer_science%29>
<https://docs.python.org/3/library/struct.html>
<https://docs.python.org/3/tutorial/inputoutput.html>
<https://pyserial.readthedocs.io/en/latest/shortintro.html>



In [ ]:
from embsec import Serial

def send_large_file():
    ser = Serial("/embsec/io_python/send_large_file")
    # Your code goes here!
    with open ('large_file.bin','r') as file:
        filecon = file.read()
    for i in range(0,len(filecon),14):
        frame_data = filecon[i:i+14]
        frame_size=len(frame_data)
        frame=struct.pack('>h14s',frame_size,frame_data.encode())
        ser.write(frame)
    return(ser.read_until())
send_large_file()


Process SendLargeFile-8:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/site-packages/core/challenge.py", line 73, in run
    self.grader(*self.grader_pttys)
  File "/opt/conda/lib/python3.7/site-packages/embsec/lessons/io_python/io_python.py", line 140, in send_large_file_grader
    frame_size, = struct.unpack("<H", ser.read(1))
struct.error: unpack requires a buffer of 2 bytes
